In [ ]:
## This file is to replicate voc training and evaluation...want to see ins and outs

In [ ]:

%pylab inline
%load_ext autoreload
%autoreload 2

import os
import sys
sys.argv=['']
sys.path.append('../../')


import numpy as np
import utils.helpers as helpers
import eva_storage.evaluation.evaluate_ssd as evaluate_ssd
from loaders.uadetrac_loader import UADetracLoader

In [ ]:
## Starting modification of train_ssd.py for ua-detrac
## importing all relevant files
import argparse
import os
import logging
import sys
import itertools
import torch
import config as jaeho_config
from torch.utils.data import DataLoader, ConcatDataset
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR


from eva_storage.external.ssd.vision.utils.misc import str2bool, Timer, freeze_net_layers, store_labels
from eva_storage.external.ssd.vision.ssd.ssd import MatchPriorModified, MatchPrior
from eva_storage.external.ssd.vision.ssd.vgg_ssd import create_vgg_ssd
from eva_storage.external.ssd.vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd
from eva_storage.external.ssd.vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite
from eva_storage.external.ssd.vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite
from eva_storage.external.ssd.vision.ssd.vgg_ssd import create_vgg_ssd
from eva_storage.external.ssd.vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite
from eva_storage.external.ssd.vision.datasets.voc_dataset import VOCDataset
#from eva_storage.external.ssd.vision.datasets.open_images import OpenImagesDataset
from eva_storage.external.ssd.vision.nn.multibox_loss import MultiboxLoss
from eva_storage.external.ssd.vision.ssd.config import vgg_ssd_config
from eva_storage.external.ssd.vision.ssd.config import mobilenetv1_ssd_config
from eva_storage.external.ssd.vision.ssd.config import squeezenet_ssd_config
from eva_storage.external.ssd.vision.ssd.data_preprocessing import TrainAugmentation, TestTransform



DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    logging.info("Use Cuda.")


In [ ]:
#dataset_path = "/home/jbang36/data/VOCdevkit/VOC2007"
#validation_path = "/home/jbang36/data/VOCdevkit/VOC2007"
base_net = os.path.join("/nethome/jbang36/eva/eva_storage/external/ssd", "models/vgg16_reducedfc.pth")
batch_size = 24
num_workers = 4
num_epochs = 100
checkpoint_folder = '/nethome/jbang36/eva/eva_storage/external/ssd/models/voc'
lr = 1e-3
momentum = 0.9
weight_decay =5e-4
validation_epochs = 5
debug_steps = 100

timer = Timer()

In [ ]:
create_net = create_vgg_ssd
config = vgg_ssd_config

train_transform = TrainAugmentation(config.image_size, config.image_mean,  
                                    config.image_std)
target_transform = MatchPrior(config.priors, config.center_variance,
                                  config.size_variance, 0.5)

test_transform = TestTransform(config.image_size, config.image_mean, 
                               config.image_std)

    

## Training step

In [ ]:

def train(loader, net, criterion, optimizer, device, debug_steps=100, epoch=-1):
    net.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    for i, data in enumerate(loader):
        images, gt_locations, labels = data
        images = images.to(device)
        gt_locations = gt_locations.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        confidence, locations = net(images)
        ## need to make sure criterion takes in locations
        regression_loss, classification_loss = criterion(confidence, locations, labels, gt_locations)
        loss = regression_loss + classification_loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
        if i and i % debug_steps == 0:
            avg_loss = running_loss / debug_steps
            avg_reg_loss = running_regression_loss / debug_steps
            avg_clf_loss = running_classification_loss / debug_steps
            logging.info(
                f"Epoch: {epoch}, Step: {i}, " +
                f"Average Loss: {avg_loss:.4f}, " +
                f"Average Regression Loss {avg_reg_loss:.4f}, " +
                f"Average Classification Loss: {avg_clf_loss:.4f}"
            )
            running_loss = 0.0
            running_regression_loss = 0.0
            running_classification_loss = 0.0
            
            
    if epoch % 10 == 0:
        model_path = os.path.join(checkpoint_folder, f"ssd_voc-Epoch-{epoch}-Loss-{avg_loss}.pth")
        net.save(model_path)
        print(f"Model saved for {epoch} at directory: {model_path}")
            


def test(loader, net, criterion, device):
    net.eval()
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            confidence, locations = net(images)
            regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)
            loss = regression_loss + classification_loss

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
    return running_loss / num, running_regression_loss / num, running_classification_loss / num


In [ ]:
datasets = []
dataset_type = 'voc'
dataset_path = '/nethome/jbang36/data/VOCdevkit/VOC2007'
dataset = VOCDataset(dataset_path, transform=train_transform,
                     target_transform=target_transform)
label_file = os.path.join(checkpoint_folder, "voc-model-labels.txt")
store_labels(label_file, dataset.class_names)
num_classes = len(dataset.class_names)
validation_dataset = '/nethome/jbang36/data/VOCdevkit/VOC2007'
print("number of classes are", num_classes) ### number of classes include BACKGROUND!!!!!
print(dataset.class_names)
    
datasets.append(dataset)


train_dataset = ConcatDataset(datasets)
train_loader = DataLoader(train_dataset, batch_size,
                          num_workers=num_workers,
                          shuffle=True)


val_dataset = VOCDataset(validation_dataset, transform=test_transform,
                             target_transform=target_transform, is_test=True)
val_loader = DataLoader(val_dataset, batch_size,
                        num_workers=num_workers,
                        shuffle=False)


net = create_net(num_classes)
min_loss = -10000.0
last_epoch = -1

base_net_lr = lr
extra_layers_lr = lr

freeze_net_layers(net.base_net)
params = itertools.chain(net.source_layer_add_ons.parameters(), net.extras.parameters(),
                         net.regression_headers.parameters(), net.classification_headers.parameters())
params = [
    {'params': itertools.chain(
        net.source_layer_add_ons.parameters(),
        net.extras.parameters()
    ), 'lr': extra_layers_lr},
    {'params': itertools.chain(
        net.regression_headers.parameters(),
        net.classification_headers.parameters()
    )}
]


net.init_from_base_net(base_net)
#net.init_from_pretrained_ssd(args.pretrained_ssd)


net.to(DEVICE)

criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3,
                         center_variance=0.1, size_variance=0.2, device=DEVICE)
optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum,
                            weight_decay=weight_decay)



milestones = [int(v.strip()) for v in "80,100".split(",")]
scheduler = MultiStepLR(optimizer, milestones=milestones,
                                                 gamma=0.1, last_epoch=last_epoch)



In [ ]:
## iterate through the train dataset to see how the boxes are labeled

for i, data in enumerate(train_loader):
    images, gt_locations, labels = data
    print(images.size())
    print(gt_locations.size())
    print(gt_locations[0])
    print(labels.size())
    print(labels[0])
    break
    

In [ ]:


for epoch in range(last_epoch + 1, num_epochs):
    scheduler.step()
    train(train_loader, net, criterion, optimizer,
          device=DEVICE, debug_steps=debug_steps, epoch=epoch)
    """
    if epoch % validation_epochs == 0 or epoch == num_epochs - 1:
        val_loss, val_regression_loss, val_classification_loss = test(val_loader, net, criterion, DEVICE)
        logging.info(
            f"Epoch: {epoch}, " +
            f"Validation Loss: {val_loss:.4f}, " +
            f"Validation Regression Loss {val_regression_loss:.4f}, " +
            f"Validation Classification Loss: {val_classification_loss:.4f}"
        )
        model_path = os.path.join(checkpoint_folder, f"{net}-Epoch-{epoch}-Loss-{val_loss}.pth")
        net.save(model_path)
    """ 

### Evaluation



In [ ]:
import eva_storage.external.ssd.eval_ssd as eval_ssd
from eva_storage.external.ssd.vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor


In [ ]:


eval_dir = '/nethome/jbang36/eva/eva_storage/evaluation/voc_eval_results'
voc_dataset = '/nethome/jbang36/data/VOCdevkit/VOC2007'
#trained_model = '/nethome/jbang36/eva/others/pytorch-ssd/models/mb1-ssd-Epoch-49-Loss-3.702120249973979.pth'
trained_model = '/nethome/jbang36/eva/eva_storage/external/ssd/models/voc/ssd_voc-Epoch-140-Loss-4.849786701202393.pth'

In [ ]:
import pathlib

eval_path = pathlib.Path(eval_dir)
eval_path.mkdir(exist_ok=True)
timer = Timer()
class_names = [name.strip() for name in open(label_file).readlines()]


dataset = VOCDataset(voc_dataset, is_test=True)

true_case_stat, all_gb_boxes, all_difficult_cases = eval_ssd.group_annotation_by_class(dataset)

net = create_vgg_ssd(len(class_names), is_test=True)

timer.start("Load Model")
net.load(trained_model)
net = net.to(DEVICE)
print(f'It took {timer.end("Load Model")} seconds to load the model.')

predictor = create_vgg_ssd_predictor(net, nms_method="hard", device=DEVICE)



In [ ]:
#####################################################################################
## need to examine all_gb_boxes - is this corner form, center form, or locations?? ##
#####################################################################################

## all the loaders will be given in location format.... is the iou thing computed on locations??
## ??

for i in range(len(dataset)):
    image = dataset.get_image(i)
    boxes, labels, probs = predictor.predict(image) ## I am assuming these are locations????????
    print(boxes.size())
    print(boxes[:20])
    break




In [ ]:

results = []
for i in range(len(dataset)):
    print("process image", i)
    timer.start("Load Image")
    image = dataset.get_image(i)
    print("Load Image: {:4f} seconds.".format(timer.end("Load Image")))
    timer.start("Predict")
    boxes, labels, probs = predictor.predict(image)
    print("Prediction: {:4f} seconds.".format(timer.end("Predict")))
    indexes = torch.ones(labels.size(0), 1, dtype=torch.float32) * i
    results.append(torch.cat([
        indexes.reshape(-1, 1),
        labels.reshape(-1, 1).float(),
        probs.reshape(-1, 1),
        boxes + 1.0  # matlab's indexes start from 1
    ], dim=1))
results = torch.cat(results)


In [ ]:
print(eval_path)

In [ ]:
for class_index, class_name in enumerate(class_names):
    if class_index == 0: continue  # ignore background
    prediction_path = eval_path / f"det_test_{class_name}.txt"
    with open(prediction_path, "w") as f:
        sub = results[results[:, 1] == class_index, :]
        for i in range(sub.size(0)):
            prob_box = sub[i, 2:].numpy()
            image_id = dataset.ids[int(sub[i, 0])]
            print(
                image_id + " " + " ".join([str(v) for v in prob_box]),
                file=f
            )
aps = []
print("\n\nAverage Precision Per-class:")


In [ ]:
iou_threshold = 0.5

In [ ]:
for class_index, class_name in enumerate(class_names):
    if class_index == 0:
        continue
    prediction_path = eval_path / f"det_test_{class_name}.txt"
    ap = eval_ssd.compute_average_precision_per_class(
        true_case_stat[class_index],
        all_gb_boxes[class_index],
        all_difficult_cases[class_index],
        prediction_path,
        iou_threshold,
        True
    )
    aps.append(ap)
    print(f"{class_name}: {ap}")

print(f"\nAverage Precision Across All Classes:{sum(aps)/len(aps)}")